In [485]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=4, suppress=True)

In [486]:
from matplotlib.patches import Ellipse    
        
def plot_gaussian(mus, sigmas, ax, weights = None):
    K = int(mus.shape[0])
    #K = 2
    if weights is None:
        weights = np.ones(K)/K
    
    for k in range(K):
        sigma = sigmas[k]
        val,vec = np.linalg.eig(sigma)
        angle = np.arctan2(vec[0,1], vec[0,0])
        ells = Ellipse(xy=mus[k],
                width=4.*np.sqrt(val[0]), height=4.*np.sqrt(val[1]),
                angle=np.rad2deg(angle))
        #ells.set_clip_box(a.bbox)
        ells.set_alpha(weights[k])
        ax.add_artist(ells)
    #plt.xlim(-20, 20)
    #plt.ylim(-20, 20)
    return

In [762]:
import numpy as np
import matplotlib.pyplot as plt
D = 25
K = 5
K_true = 2
N = 40000
is_plot = False

mus_true = []
sigmas_true = []
samples = []
for k in range(K_true):
    mu = 5*np.random.normal(size=D)
    sigma = 3*np.diag(np.random.rand(D))
    sample = np.random.multivariate_normal(mu, cov = sigma, size = N/K_true)
    mus_true.append(mu)
    sigmas_true.append(sigma)
    samples.append(sample)
    
x = np.vstack(samples)
mus_true = np.array(mus_true)
sigmas_true = np.array(sigmas_true)

if is_plot:
    fig,ax = plt.subplots()
    plot_gaussian(mus_true,sigmas_true,ax)
    plt.plot(x[:,0],x[:,1],'*')

### Define the parameters 

In [763]:
#pi params
alpha0 = np.ones(K)*0.1
alpha = alpha0

#mu and sigma params
betha0 = 1
betha = np.ones(K)
mu0 = np.zeros(D)
W0 = np.eye(D)
v0 = D
mu = x[np.random.choice(len(x),size = K)]
W = [np.eye(D) for i in range(K)]
v = np.ones(K)*D

#

In [764]:
from scipy.special import digamma
from scipy.special import logsumexp

### expectation step

In [765]:
def log_normalize(x):
    return x - logsumexp(x)

In [766]:
def expectation(alpha, betha, mu, W, v, x):
    ln_ro = np.zeros([N,K])
    ro = np.zeros([N,K])
    
    print 'Calculating ro'
    tic = time.time()
    for k in range(K):
        E_s = D*np.log(2) + np.log(np.linalg.det(W[k])) + np.sum([digamma((v[k] + 1 - i)/2.) for i in range(D)])
        E_pi = digamma(alpha[k]) - digamma(np.sum(alpha))
        E_2 = D*np.log(2*np.pi)
        for n in range(N):
            E_ms = D/betha[k] + v[k]*np.dot(x[n] - mu[k], np.dot(W[k], x[n] - mu[k]))
            ln_ro[n,k] = E_pi + 0.5*E_s - 0.5*E_2 - 0.5*E_ms
            
    for n in range(N):        
        ln_ro[n,:] = log_normalize(ln_ro[n,:])
        ro[n,:] = np.exp(ln_ro[n,:])
    toc = time.time()
    print toc-tic

    print 'Calculating Nk'
    tic = time.time()
    Nks = np.array([np.sum(ro[:,k]) for k in range(K)])
    toc = time.time()
    print toc-tic
    

    print 'Calculating xk'
    xks = np.dot(ro.T, x)
    for k in range(K):
        xks[k,:] /= Nks[k]
    toc = time.time()
    print toc-tic
    
    print 'Calculating Sk' 
    Sks = []
    for k in range(K):
        Sk = np.zeros([D,D])
        for n in range(N):
            Sk += ro[n,k]*np.outer(x[n]-xks[k], x[n]-xks[k])
        Sk /= Nks[k]
        Sks.append(Sk)
    
    toc = time.time()
    print toc-tic
    
    
    return ro, ln_ro, Nks, xks, Sks

def maximization(Nks, xks, Sks, ro):
    alpha = alpha0 + Nks
    betha = betha0 + Nks

    for k in range(K):
        mu[k] = (betha0*mu0 + Nks[k]*xks[k])/betha[k]
        Wk_inv = np.linalg.inv(W0) + Nks[k]*Sks[k] + np.outer(xks[k]-mu0, xks[k]-mu0)*betha0*Nks[k]/(betha0+Nks[k])
        W[k] = np.linalg.inv(Wk_inv)
        v[k] = v0 + Nks[k]
    
    return alpha, betha, mu, W, v


In [767]:
import time

for i in range(30):
    print 'Iteration ' + str(i)
    print 'Expectation'
    tic = time.time()
    ro, ln_ro, Nks, xks, Sks = expectation(alpha, betha, mu, W, v, x)
    toc = time.time()
    print toc-tic

    print 'Maximization'
    tic = time.time()
    alpha, betha, mu, W, v = maximization(Nks, xks, Sks, ro)
    toc = time.time()
    print toc-tic


Iteration 0
Expectation
Calculating ro
1.92328000069
Calculating Nk
0.000528812408447
Calculating xk
0.00169491767883
Calculating Sk
1.53369188309
3.45758199692
Maximization
0.00886702537537
Iteration 1
Expectation
Calculating ro
1.92752814293
Calculating Nk
0.000401020050049
Calculating xk
0.00179195404053
Calculating Sk
1.57078289986
3.49878096581
Maximization
0.00358700752258
Iteration 2
Expectation
Calculating ro
1.93645715714
Calculating Nk
0.000393867492676
Calculating xk
0.00168395042419
Calculating Sk
1.57053685188
3.50750803947
Maximization
0.00387597084045
Iteration 3
Expectation
Calculating ro
1.95281505585
Calculating Nk
0.000488042831421
Calculating xk
0.0016930103302
Calculating Sk
1.62337803841
3.57688593864
Maximization
0.00223088264465
Iteration 4
Expectation
Calculating ro
1.94130301476
Calculating Nk
0.000243902206421
Calculating xk
0.00125885009766
Calculating Sk
1.56788086891
3.51033687592
Maximization
0.00916504859924
Iteration 5
Expectation
Calculating ro
1.99082

In [773]:
sigmas = [np.linalg.inv(v[k]*W[k]) for k in range(K)]
pis = alpha/np.sum(alpha)

In [775]:
print pis

[0.5    0.0005 0.0202 0.1482 0.3312]


In [776]:
if is_plot:
    fig,ax = plt.subplots()
    plt.plot(x[:,0],x[:,1],'r*',zorder=1)
    plot_gaussian(mu,sigmas,ax, pis)

In [777]:
for k in range(K):
    if pis[k] > 0.1: 
        print pis[k]
        print mu[k]

0.49999625004687437
[-8.6683  2.7609 -6.7305 -0.9055  3.5907  0.0278  1.7871  4.417  -2.8146
 -4.6792  2.6002  1.8998  0.4251  1.5035  0.113  -5.033  -1.7115 -1.4696
 -2.5152 -2.1759  2.6834 -0.9074 -1.1848 -2.1309 -4.404 ]
0.14822036546804138
[ -1.8696   9.6085  -5.5255   4.953   -0.6253   0.5476  -6.9743  -1.4933
  -5.8154  -1.5329   4.0315  -3.0967 -11.5813  -5.1405   9.6936  -4.4168
  11.7063   5.2548  -5.7146   6.6657   0.0632  -0.2553  -4.1933   4.6598
  -3.2653]
0.33116025510115393
[ -2.3487   9.1882  -5.6359   4.9018  -0.583    0.9416  -6.9506  -1.6298
  -6.4834  -1.6711   3.7991  -2.9316 -11.5929  -5.0587   9.8182  -4.4168
  11.694    5.2587  -5.7958   7.5007   0.0516  -0.2421  -4.3261   4.6629
  -3.0183]


In [778]:
print mus_true

[[ -8.6586   2.7586  -6.7332  -0.9139   3.5962   0.0184   1.7815   4.4221
   -2.8126  -4.6787   2.5914   1.8995   0.4284   1.5045   0.1138  -5.0166
   -1.6954  -1.4696  -2.5163  -2.1988   2.6922  -0.9087  -1.184   -2.1253
   -4.3999]
 [ -2.2008   9.3413  -5.605    4.9371  -0.6107   0.7984  -6.9538  -1.5919
   -6.2386  -1.6245   3.8638  -3.0214 -11.5973  -5.0893   9.7896  -4.4131
   11.699    5.2599  -5.7691   7.2315   0.0535  -0.2484  -4.2894   4.6499
   -3.0538]]


In [780]:
from sklearn.mixture import BayesianGaussianMixture

In [781]:
BGMM = BayesianGaussianMixture(n_components=10)

In [782]:
BGMM.fit(x)

/home/teguh/.local/lib/python2.7/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


BayesianGaussianMixture(covariance_prior=None, covariance_type='full',
            degrees_of_freedom_prior=None, init_params='kmeans',
            max_iter=100, mean_precision_prior=None, mean_prior=None,
            n_components=10, n_init=1, random_state=None, reg_covar=1e-06,
            tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
            weight_concentration_prior=None,
            weight_concentration_prior_type='dirichlet_process')

In [783]:
print BGMM.means_

[[ -2.5281   9.0432  -5.8755   4.7392   0.0327   0.8485  -6.6504  -2.7118
   -5.1207  -1.4534   4.1627  -2.5522 -11.4505  -4.803   10.4236  -5.2035
   11.1941   4.5136  -5.8104   8.4205   0.2326   0.5783  -4.1306   2.9915
   -3.3718]
 [ -8.7169   2.5771  -6.8514  -1.0315   3.5035  -0.1349   1.9672   4.4479
   -2.7826  -4.6818   2.6777   1.9556   0.5577   1.4828   0.1491  -4.7453
   -1.622   -1.4874  -2.5327  -2.2941   2.8408  -0.8881  -1.3153  -2.0006
   -4.4345]
 [ -2.3086   9.6206  -5.2136   4.4498  -0.8813   1.186   -6.9396  -2.7282
   -6.8626  -1.9792   2.9946  -3.0447 -11.1489  -4.7954   9.0191  -4.1283
   11.4578   5.2319  -5.2805   6.3534   0.206   -0.7365  -5.2885   4.0874
   -4.0327]
 [ -8.6103   2.9824  -6.5862  -0.7539   3.6952   0.2231   1.5702   4.3798
   -2.8536  -4.6762   2.5077   1.8326   0.2649   1.5278   0.0708  -5.3787
   -1.8176  -1.4477  -2.4948  -2.0333   2.4946  -0.9305  -1.0287  -2.2866
   -4.3678]
 [ -2.1667   9.6927  -5.3545   3.7791  -1.0687  -0.0499  -6.9872

In [784]:
BGMM.weights_

array([0.0005, 0.2727, 0.0006, 0.2273, 0.0005, 0.2587, 0.2379, 0.0007,
       0.0006, 0.0006])

In [785]:
for k in range(10):
    if BGMM.weights_[k] > 0.1:
        print BGMM.means_[k]

[-8.7169  2.5771 -6.8514 -1.0315  3.5035 -0.1349  1.9672  4.4479 -2.7826
 -4.6818  2.6777  1.9556  0.5577  1.4828  0.1491 -4.7453 -1.622  -1.4874
 -2.5327 -2.2941  2.8408 -0.8881 -1.3153 -2.0006 -4.4345]
[-8.6103  2.9824 -6.5862 -0.7539  3.6952  0.2231  1.5702  4.3798 -2.8536
 -4.6762  2.5077  1.8326  0.2649  1.5278  0.0708 -5.3787 -1.8176 -1.4477
 -2.4948 -2.0333  2.4946 -0.9305 -1.0287 -2.2866 -4.3678]
[ -2.2029   9.4989  -5.5746   4.9731  -0.615    0.7753  -6.9537  -1.3412
  -6.3943  -1.6764   3.7618  -2.8942 -11.5994  -5.1104   9.9226  -4.3848
  11.677    5.2693  -5.7898   7.4339  -0.0003  -0.2931  -4.3407   4.2765
  -3.1754]
[ -2.2148   9.1375  -5.6216   4.8726  -0.5758   0.8181  -6.9559  -1.8774
  -6.0622  -1.5762   3.9611  -3.1544 -11.58    -5.0579   9.5997  -4.4633
  11.7248   5.2407  -5.7507   7.0038   0.116   -0.1975  -4.2333   5.104
  -2.9204]


In [786]:
print mus_true

[[ -8.6586   2.7586  -6.7332  -0.9139   3.5962   0.0184   1.7815   4.4221
   -2.8126  -4.6787   2.5914   1.8995   0.4284   1.5045   0.1138  -5.0166
   -1.6954  -1.4696  -2.5163  -2.1988   2.6922  -0.9087  -1.184   -2.1253
   -4.3999]
 [ -2.2008   9.3413  -5.605    4.9371  -0.6107   0.7984  -6.9538  -1.5919
   -6.2386  -1.6245   3.8638  -3.0214 -11.5973  -5.0893   9.7896  -4.4131
   11.699    5.2599  -5.7691   7.2315   0.0535  -0.2484  -4.2894   4.6499
   -3.0538]]
